In [1]:
import transformers as ts

d:\tensorGo-task\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
import datasets
dataset = datasets.load_dataset("google-research-datasets/go_emotions")

In [4]:
tokenizer = ts.AutoTokenizer.from_pretrained('google-bert/bert-base-cased')

In [5]:
import torch
from transformers import Trainer, TrainingArguments
from accelerate import Accelerator

print("PyTorch version:", torch.__version__)
print("Transformers version:", TrainingArguments.__module__)
print("Accelerate available:", Accelerator is not None)


PyTorch version: 2.5.1+cu124
Transformers version: transformers.training_args
Accelerate available: True


In [6]:
import accelerate

accelerate.__version__

'1.1.1'

In [7]:
args = ts.TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
)

In [8]:
model = ts.AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased",num_labels=28)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def preprocess_function(examples):
    tokenized_output = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_output['labels'] = [label[0] if len(label) == 1 else max(label) for label in examples['labels']]
    
    return tokenized_output

In [10]:
tokenized_test = dataset['test'].map(preprocess_function,batched=True)
tokenized_val = dataset['validation'].map(preprocess_function,batched=True)
tokenized_train = dataset['train'].map(preprocess_function,batched=True)

Map: 100%|██████████| 43410/43410 [00:10<00:00, 4295.74 examples/s]


In [11]:
print(tokenized_train['labels'][0:10])


[27, 27, 2, 14, 3, 26, 15, 20, 0, 27]


In [12]:
trainer = ts.Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

In [13]:
trainer.train()

  3%|▎         | 500/16281 [06:11<3:29:47,  1.25it/s]

{'loss': 2.2142, 'grad_norm': 21.305633544921875, 'learning_rate': 4.8464467784534126e-05, 'epoch': 0.09}


  6%|▌         | 1000/16281 [13:00<3:35:45,  1.18it/s]

{'loss': 1.8103, 'grad_norm': 17.060583114624023, 'learning_rate': 4.6928935569068236e-05, 'epoch': 0.18}


  9%|▉         | 1500/16281 [20:02<3:32:28,  1.16it/s]

...

 98%|█████████▊| 16000/16281 [3:12:41<03:13,  1.45it/s]

{'loss': 0.7476, 'grad_norm': 5.493292808532715, 'learning_rate': 8.629691050918249e-07, 'epoch': 2.95}


                                                       
100%|██████████| 16281/16281 [3:18:16<00:00,  1.37it/s]

{'eval_loss': 1.546319603919983, 'eval_runtime': 139.2463, 'eval_samples_per_second': 38.967, 'eval_steps_per_second': 4.876, 'epoch': 3.0}
{'train_runtime': 11896.8525, 'train_samples_per_second': 10.947, 'train_steps_per_second': 1.369, 'train_loss': 1.227757647160369, 'epoch': 3.0}


TrainOutput(global_step=16281, training_loss=1.227757647160369, metrics={'train_runtime': 11896.8525, 'train_samples_per_second': 10.947, 'train_steps_per_second': 1.369, 'total_flos': 3.427295167033344e+16, 'train_loss': 1.227757647160369, 'epoch': 3.0})

In [14]:
trainer.save_model("./emotion_detect")